In [82]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [83]:
import pandas as pd
csv_file_path = '/kaggle/input/rainfall-in-india/rainfall in india 1901-2015.csv'
df = pd.read_csv(csv_file_path)
bihar_df = df[df['SUBDIVISION'] == 'BIHAR'][["YEAR","ANNUAL"]]
bihar_df = bihar_df.reset_index(drop=True)

In [84]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [85]:
df = bihar_df
bihar_df = bihar_df[:5]

In [86]:
years = np.array(bihar_df["YEAR"])
rainfall = np.array(bihar_df["ANNUAL"])
scaler = MinMaxScaler(feature_range=(0, 1))
rainfall_normalized = scaler.fit_transform(rainfall.reshape(-1, 1))
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)
seq_length = 3 
X, y = create_sequences(rainfall_normalized, seq_length)

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [88]:
model.fit(X, y, epochs=15, batch_size=32)

Epoch 1/15
1/1 [==============================] - 5s 5s/step - loss: 0.6738
Epoch 2/15
1/1 [==============================] - 0s 10ms/step - loss: 0.6576
Epoch 3/15
1/1 [==============================] - 0s 9ms/step - loss: 0.6385
Epoch 4/15
1/1 [==============================] - 0s 10ms/step - loss: 0.6253
Epoch 5/15
1/1 [==============================] - 0s 10ms/step - loss: 0.5991
Epoch 6/15
1/1 [==============================] - 0s 9ms/step - loss: 0.5863
Epoch 7/15
1/1 [==============================] - 0s 8ms/step - loss: 0.5697
Epoch 8/15
1/1 [==============================] - 0s 9ms/step - loss: 0.5659
Epoch 9/15
1/1 [==============================] - 0s 9ms/step - loss: 0.5487
Epoch 10/15
1/1 [==============================] - 0s 9ms/step - loss: 0.5456
Epoch 11/15
1/1 [==============================] - 0s 9ms/step - loss: 0.4427
Epoch 12/15
1/1 [==============================] - 0s 8ms/step - loss: 0.4978
Epoch 13/15
1/1 [==============================] - 0s 9ms/step - loss: 

In [89]:
n_future = 5  # number of years you want to predict

forecast_period_dates = np.array([years[-1] + i for i in range(1, n_future + 1)])
forecast = rainfall_normalized[-seq_length:].tolist()
for _ in range(n_future):
    x = np.array(forecast[-seq_length:]).reshape(1, seq_length, 1)
    out = model.predict(x)[0][0]
    forecast.append([out])
forecast = scaler.inverse_transform(np.array(forecast[seq_length:])).reshape(-1)

1/1 [==============================] - 0s 21ms/step


In [90]:
forecast

array([1100.864 , 1121.2196, 1114.7915, 1083.0715, 1083.0935],
      dtype=float32)

In [91]:
bihar_df

,YEAR,ANNUAL
0,1901,952.3
1,1902,1206.2
2,1903,1016.3
3,1904,1269.5
4,1905,1498.5
